In [1]:
import pandas as pd
import numpy as np
import itertools
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [2]:
# The matrix is constructed by proceeding row by row, starting from the top row (0)
# lij choice : we calculate G of the term (li0 li1 ... lij) which is equal to the number of examples in class 1 - the number of examples in class 0 that satisfy this term
# and we take the lij that gives the greatest G.
# The chosen lij must differ from [li0 li1 ... li(j-1)] and their negations.
# Also, The chosen lij must not be among literals of rows [0 ... i-1], columns [0 ... limite-1]
# k <= nb_features
def Construct_matrix (k,features,class_name,data):
    matrix = pd.DataFrame([[[] for _ in range(k)] for _ in range(k)])
    nbf = len(features)
    for i in range(k):
        for j in range(k): 
            ml = matrix.iloc[i, :j].tolist()
            mf = [f for f, _ in ml]
            mlc1 = ml + [[class_name,1]]
            mlc0 = ml + [[class_name,0]]
            m=0
            tabG = pd.DataFrame(columns= ['feature','value','g'])
            for f in [item for item in features if item not in mf]:
                if (i==0):
                    limite = 0
                else :
                    limite = (2*nbf - 2*j)/i
                    if limite > k-j :
                        limite = k-j
                    else:
                        if int(limite) == limite :
                            limite = int(limite-1)
                        else:
                            limite = int (limite)
                ll = matrix.iloc[0:i, 0:limite].values.flatten().tolist()
                index = [i for i, l in enumerate(ll) if l[0] == f]
                f0 = [ind for ind in index if ll[ind][1] == 0]
                f1 = [ind for ind in index if ll[ind][1] == 1]
                if not f1:
                    mlfc1 = mlc1 +  [[f,1]]
                    mlfc0 = mlc0 +  [[f,1]]
                    tabG.loc[m] = [f,1,( len(data.query(' & '.join([f'{col} == {val}' for col, val in mlfc1]))) - len(data.query(' & '.join([f'{col} == {val}' for col, val in mlfc0]))) ) ]  
                    m=m+1
                if not f0:
                    mlfc1 = mlc1 +  [[f,0]]
                    mlfc0 = mlc0 +  [[f,0]]
                    tabG.loc[m] = [f,0,( len(data.query(' & '.join([f'{col} == {val}' for col, val in mlfc1]))) - len(data.query(' & '.join([f'{col} == {val}' for col, val in mlfc0]))) )] 
                    m=m+1
            r = tabG[tabG['g']==tabG['g'].max()].sample(n=1)
            matrix.iloc[i,j] = [r.iloc[0,0],r.iloc[0,1]]
    return matrix

# Consider term by term, starting with terms of size k  and decreasing down to 1
# The term must be consistent (i.e. it does not contain both a literal and its negation)
# Calculate P and Q , where P (respectively, Q) represents the number of examples in class 1 (respectively, class 0) that satisfy this term and are not already covered by the selected terms.
# Take this term if (P != 0 and q < p) OR if this term does not cover any example from class 0
# Stop if all examples in class 1 are covered or there are no more terms to evaluate
def Construct_kdnf (matrix,k,features,data,Ydata):
    all_e_c1 = []
    all_e_c0 = []
    terms=[]
    ks = k+1 
    nb_e_c1 = Ydata.sum()
    #Keep in a dataframe for each term taken, the list of examples covered by this term
    tab_exp = pd.DataFrame(columns= ['term','e_c1','e_c0'])
    while ( len(all_e_c1) < nb_e_c1 ) and ks != 1 :
        ks = ks - 1 
        rpi_k = list(sorted(filter(lambda m: sum(m) == ks, itertools.product(range(k + 1), repeat=k)),  key=lambda x: (max(x), x), reverse=True))
        r=0
        while  ( len(all_e_c1) < nb_e_c1 )  and ( r < len(rpi_k) ):
            stop = False
            while ( ( r < len(rpi_k) ) and (stop == False)):
                stop = True
                rpi = rpi_k[r]
                r=r+1
                d={}
                for i in range(k):
                    for j in range(rpi[i]):
                        if (matrix.iloc[i,j][0]) in d :
                            if d[(matrix.iloc[i,j][0])] != matrix.iloc[i,j][1] :
                                stop = False
                                break
                        else:
                            d[(matrix.iloc[i,j][0])] = matrix.iloc[i,j][1]
                    if stop == False:
                        break
                if (stop==True) :
                    if d in terms :
                        stop = False
            if (stop==True) :
                ec1 =[]
                ec0 =[]
                terms.append(d)
                dt = data.query(' & '.join([f'{f} == {l}' for f,l in d.items()]))
                for i in dt.index :
                    if ( Ydata[i] == 1 ) :
                        ec1.append(i)
                    else:
                        ec0.append(i)
                p = len(set(ec1) - set(all_e_c1))
                q = len(set(ec0) - set(all_e_c0))
                if ( p!=0 and (q<p) ) or  len(ec0) == 0 :
                    tab_exp.loc[len(tab_exp)] = [d,ec1,ec0]   
                    all_e_c1 = list(set(all_e_c1) | set(ec1))
                    all_e_c0 = list(set(all_e_c0) | set(ec0))
    return  tab_exp,all_e_c1,all_e_c0

# For each term, determine whether to remove it or not
def pruning_kdnf(tab_exp):
    for index, row in tab_exp.iterrows():
        t,ec1,ec0 = row
        for index_2, row_2 in tab_exp.iterrows():
            if index_2 != index :
                t_2,ec1_2,ec0_2 = row_2
                ec1 = list(set(ec1) - set(ec1).intersection(set(ec1_2)))
                ec0 = list(set(ec0) - set(ec0).intersection(set(ec0_2)))
                if len(ec1) == 0 :
                    tab_exp = tab_exp.drop(index)
                    break            
        if ( len(ec1) != 0 ) and (len(ec0)>=len(ec1)) :
            tab_exp = tab_exp.drop(index)
    return tab_exp
    
#Write a term (using features,not,and)
def write_term (literals):
    term = ""
    for f,l in literals.items():
        if term =="" :
            term += f"{f}" if l == 1 else f"not {f}"
        else :
            term += f" and not {f}" if l == 0 else f" and {f}"
    return term

#Write the kdnf (term1 or term2 .... or termm)
def write_kdnf (tab_exp):
    exp =""
    for index,row in tab_exp.iterrows():
        t,ec1,ec0 = row
        if  exp == "" :
            tm = write_term(t)
            exp += "("+ tm +")"
        else:
            tm = write_term(t)
            exp += " or ("+ tm +")"
    return exp

#Returns the class of an instance
def eval_kdnf (kdnf,literals):
    if not kdnf :
        return 0
    else :
        return int(eval(kdnf, {},literals ))

#Returns the accuracy of the kndf
def calcul_accuracy (kdnf,Xdata,Ydata):
    v=0
    for i, row in Xdata.iterrows():
        if eval_kdnf(kdnf,Xdata.loc[i]) == Ydata[i]:
            v = v+1
    return (v / len(Xdata))*100

In [3]:
#First simple example of test : the full truth-table of (a ∧ b) ∨ (c ∧ d)
data = pd.read_csv("../data/data-ab+cd.csv")
features = ['fa', 'fb','fc', 'fd']
class_name = 'c'
Xdata = data[features] 
Ydata = data[class_name]
k=2
matrice = Construct_matrix(k,features,class_name,data)
tab_exp1,aec1,aec0 = Construct_kdnf(matrice,k,features,data,Ydata)
tab_exp2 = pruning_kdnf(tab_exp1)
kdnf = write_kdnf(tab_exp2)
accuracy = calcul_accuracy (kdnf,Xdata,Ydata)
print('------------------------------------------------------------------------------------------------------------')
print('------------------------------------------------- K-DNF : ---------------------------------------------------')
print ('accuracy = ',accuracy,'%')
DT = DecisionTreeClassifier(criterion='gini',max_depth=k)
DT.fit(Xdata,Ydata)
y_pred_test = DT.predict(Xdata) 
accuracy2 = accuracy_score(Ydata,y_pred_test)
print('------------------------------------------------------------------------------------------------------------')
print('-------------------------------------------------  DT  : ---------------------------------------------------')
print ('accuracy = ',accuracy2*100,'%')
print('------------------------------------------------------------------------------------------------------------')

------------------------------------------------------------------------------------------------------------
------------------------------------------------- K-DNF : ---------------------------------------------------
accuracy =  100.0 %
------------------------------------------------------------------------------------------------------------
-------------------------------------------------  DT  : ---------------------------------------------------
accuracy =  81.25 %
------------------------------------------------------------------------------------------------------------


In [4]:
#Example of test
number_BF = 11 #number of features
features = [f'F{i}' for i in range(1,(number_BF+1))]
class_name = "Class"
all_names = ["Class"]+features
train_data = pd.read_csv("../data/monks-1-binary.train",sep=' ',header=None, names=all_names)
test_data = pd.read_csv("../data/monks-1-binary.test",sep=' ',header=None, names=all_names)
#data = pd.read_csv('filename',sep=' ',header=None, names= all_names)
#train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)
x_train = train_data[features] 
x_test = test_data[features] 
y_train = train_data[class_name]
y_test = test_data[class_name]
k = 2
matrix = Construct_matrix(k,features,class_name,train_data) 
tab_exp1,aec1,aec0 =  Construct_kdnf(matrix,k,features,train_data,y_train)
tab_exp2 = pruning_kdnf(tab_exp1)
kdnf = write_kdnf(tab_exp2)
accuracy_test = calcul_accuracy(kdnf,x_test,y_test)
nb_term_kdnf = len(tab_exp2)
print('-------------------------------------------------------------------------------------------------------------')
print('------------------------------------------------- K-DNF -----------------------------------------------------')
print ('\033[31mTest Accuracy = \033[0m',round(accuracy_test,2),'%')
print('-------------------------------------------------------------------------------------------------------------')

-------------------------------------------------------------------------------------------------------------
------------------------------------------------- K-DNF -----------------------------------------------------
Test Accuracy =  75.0 %
-------------------------------------------------------------------------------------------------------------
